In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-pss5e1/test_pss5e1.csv
/kaggle/input/train-pss5e1/train_pss5e1.csv


In [2]:
train = pd.read_csv('/kaggle/input/train-pss5e1/train_pss5e1.csv')
test = pd.read_csv('/kaggle/input/test-pss5e1/test_pss5e1.csv')

train = train.dropna(subset='num_sold')

train['time_id'] = train['country'] + '_' + train['product'] + '_' + train['store']
test['time_id'] = test['country'] + '_' + test['product'] + '_' + test['store']

target = 'num_sold'
features = [col for col in train.columns if (col!='num_sold' and col!='date' and col!='id')]
cols_to_drop = ['date', 'id']
train.drop(columns=cols_to_drop, inplace=True)
test.drop(columns=cols_to_drop, inplace=True)

In [3]:
from scipy.stats import boxcox
train['num_sold'], lmbda = boxcox(train['num_sold'])

In [4]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()
h2o_df = h2o.H2OFrame(train)

y = 'num_sold'
X = [col for col in features if col != 'num_sold']

aml = H2OAutoML(
    max_models=50,                  # Maximum number of models to train
    max_runtime_secs=39600,          # Max runtime in seconds
    seed=42,                        # Seed for reproducibility
    sort_metric='RMSE',             # Evaluation metric
    nfolds=5,                       # Number of folds for cross-validation
    include_algos=["XGBoost", "DeepLearning", "StackedEnsemble", "GBM", "DRF"],  # Models to include   # Exclude CPU-heavy algorithms
)

aml.train(x=X, y=y, training_frame=h2o_df)
lb = aml.leaderboard
print(lb)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.25" 2024-10-15; OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpb6cvo9_q
  JVM stdout: /tmp/tmpb6cvo9_q/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpb6cvo9_q/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,2 months and 23 days
H2O_cluster_name:,H2O_from_python_unknownUser_06ijwo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                     rmse       mse       mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20250125_200219     0.568375  0.32305   0.416888  0.0219791                  0.32305
StackedEnsemble_BestOfFamily_1_AutoML_1_20250125_200219  0.578761  0.334964  0.424546  0.0224811                  0.334964
XGBoost_grid_1_AutoML_1_20250125_200219_model_6          0.585237  0.342502  0.432063  0.0232928                  0.342502
XGBoost_grid_1_AutoML_1_20250125_200219_model_14         0.587085  0.344669  0.431867  0.0229747                  0.344669
XGBoost_grid_1_AutoML_1_20250125_200219_model_5          0.591857  0.350295  0.434496  0.0230983                  0.350295
XGBoost_grid_1_AutoML_1_20250125_200219_model_13         0.591902  0.3503

In [5]:
leaderboard = aml.leaderboard
model_save_dir = '/kaggle/working/raw_data_models_final/'  

for model_id in leaderboard['model_id'].as_data_frame()['model_id']:
    model = h2o.get_model(model_id)
    model_path = h2o.save_model(model=model, path=model_save_dir, force=True)
    print(f"Model {model_id} saved to {model_path}")

leaderboard_df = leaderboard.as_data_frame()
leaderboard_csv_path = os.path.join(model_save_dir, 'leaderboard.csv')
leaderboard_df.to_csv(leaderboard_csv_path, index=False)
print(f"Leaderboard saved to {leaderboard_csv_path}")
h2o.shutdown()

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model StackedEnsemble_AllModels_1_AutoML_1_20250125_200219 saved to /kaggle/working/raw_data_models_final/StackedEnsemble_AllModels_1_AutoML_1_20250125_200219
Model StackedEnsemble_BestOfFamily_1_AutoML_1_20250125_200219 saved to /kaggle/working/raw_data_models_final/StackedEnsemble_BestOfFamily_1_AutoML_1_20250125_200219
Model XGBoost_grid_1_AutoML_1_20250125_200219_model_6 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20250125_200219_model_6
Model XGBoost_grid_1_AutoML_1_20250125_200219_model_14 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20250125_200219_model_14
Model XGBoost_grid_1_AutoML_1_20250125_200219_model_5 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20250125_200219_model_5
Model XGBoost_grid_1_AutoML_1_20250125_200219_model_13 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20250125_200219_model_13
Model GBM_4_AutoML_1_20250125_200219 saved to /kaggle/working/raw_data_model

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
<ipython-input-5-84012a467d30>:13: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
